In [17]:
import numpy as np
import pandas as pd
from scipy import stats
from IPython.display import Markdown

In [12]:
ALPHA = 0.05

# Множественная регрессия

# Нелинейная модель

# Письменные задачи

1\. Рассматривается регрессионная модель, построенная на основе 100 наблюдений.

Используя тест Фаррара-Глоубера определите есть ли в массиве объясняющих переменных мультиколлинеарность.

Рассчитайте F-критерии теста Фаррара-Глоубера и определите, какая переменная больше влияет на общую мультиколлинеарность.

Рассчитайте частные коэффициенты корреляции.

In [4]:
R_inv = np.array([
    [2.9, -1.043, -1.633],
    [-2.232, 2.683, -0.131],
    [-0.142, -1.415, 1.96]
])

In [14]:
R = np.linalg.inv(R_inv)
n = 100
k = 3

In [18]:
fg_value = -(n - 1 - (1 / 6) * (2 * k + 5)) * np.log(np.linalg.det(R))
print(fg_value)

chi2_critical = stats.chi2.ppf(1 - ALPHA, k * (k - 1) / 2)
print(chi2_critical)

142.86900020884826
7.814727903251179


In [27]:
Markdown(rf"""
$ FG = {fg_value:.2f} > X^2_{{крит}} = {chi2_critical:.2f} \rightarrow $ гипотеза H0 **отвергается** -
в массиве объясняющих переменных существует мультиколлинеарность.
""")


$ FG = 142.87 > X^2_{крит} = 7.81 \rightarrow $ гипотеза H0 **отвергается** -
в массиве объясняющих переменных существует мультиколлинеарность.


In [31]:
f_value = (n - k - 1) * (np.diagonal(R_inv) - 1) / k
f_value = pd.Series(f_value, index=[f'x{i}' for i in range(len(f_value))], name='F')
f_value

x0    60.800
x1    53.856
x2    30.720
Name: F, dtype: float64

In [32]:
f_critical = stats.f.ppf(1 - ALPHA, k, n - k - 1)
f_critical

2.6993925975521815

In [35]:
print(f_value > f_critical)
Markdown("""
True - исследуемая экзогенная переменная мультиколлинеарна с другими
""")

x0    True
x1    True
x2    True
Name: F, dtype: bool



True - исследуемая экзогенная переменная мультиколлинеарна с другими


In [56]:
pairs = [(0, 1), (0, 2), (1, 2), (2, 1), (2, 0), (1, 0)]
private_corr = pd.Series(dtype=object)
for i, j in pairs:
    pair = f'{i=},{j=}'
    private_corr[pair] = -R_inv[i, j] / np.sqrt(R_inv[i, i] * R_inv[j, j])

private_corr

i=0,j=1    0.373917
i=0,j=2    0.684950
i=1,j=2    0.057126
i=2,j=1    0.617047
i=2,j=0    0.059561
i=1,j=0    0.800175
dtype: float64

In [59]:
t_value = private_corr * np.sqrt(n - k - 1) / np.sqrt(1 - private_corr * private_corr)
t_value

i=0,j=1     3.950157
i=0,j=2     9.211106
i=1,j=2     0.560633
i=2,j=1     7.682800
i=2,j=0     0.584613
i=1,j=0    13.071878
dtype: float64

In [65]:
t_critical = stats.t.ppf(1 - ALPHA, n - k - 1)
t_critical

1.6608814403008005

In [71]:
summary = pd.DataFrame((abs(t_value) > t_critical).rename('bool'))  # noqa
summary['desc'] = summary['bool'].map(lambda x: 'коррел. ' + ('значима' if x else 'незначима'))
summary

,bool,desc
"i=0,j=1",True,коррел. значима
"i=0,j=2",True,коррел. значима
"i=1,j=2",False,коррел. незначима
"i=2,j=1",True,коррел. значима
"i=2,j=0",False,коррел. незначима
"i=1,j=0",True,коррел. значима


2\. В результате оценивания регрессионной модели по выборочным данным объемом 200 наблюдения получен результат.

В модели исследователь подозревает наличие мультиколлинеарности. С помощью коэффициентов вздутия проверьте данное предположение.

In [81]:
rsquared_x1__x2_x3 = 0.6
rsquared_x2__x1_x3 = 0.55
rsquared_x3__x1_x2 = 0.66

vif_list = [1 / (1 - rsquared_x1__x2_x3), 1 / (1 - rsquared_x2__x1_x3), 1 / (1 - rsquared_x3__x1_x2)]
vif = pd.DataFrame(vif_list, [f'vif_{i}' for i in range(len(vif_list))], columns=['vif'])
vif[' > 2'] = vif['vif'] > 2
vif[' < 10'] = vif['vif'] < 10
vif['desc'] = vif[' < 10'].map(lambda x: ('не ' if x else '') + 'приводит к мультиколлинеарности')
vif

,vif,> 2,< 10,desc
vif_0,2.500000,True,True,не приводит к мультиколлинеарности
vif_1,2.222222,True,True,не приводит к мультиколлинеарности
vif_2,2.941176,True,True,не приводит к мультиколлинеарности


3\. В результате оценивания регрессионной модели по выборочным данным объемом 200 наблюдения получен результат.

Вам известен определитель матрицы межфакторных корреляций (xij друг с другом) $ det(R) = 0,562 $. Используя тест Фаррара-Глоубера определите есть ли в массиве объясняющих переменных мультиколлинеарность.

In [ ]:
det_R = 0.562
n = 200
k = 3